# Heatmaps
<img src="img/heatmap-logo.png" style="float: right; padding-left: 1em;"></img>

A heatmap is a graphical representation of data where the individual values contained in a matrix are represented as colors. Be sure to normalise your data, and choose a relevant colour palette.

Heatmaps are good for showing variance across multiple variables, revealing any patterns, displaying whether any variables are similar to each other, and for detecting if any correlations exist in-between them.